In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import difflib
import spacy
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from nltk.stem.porter import *

In [2]:
data = pd.read_csv('nlp project.csv', encoding = 'utf-8').fillna(' ')

C:\Users\ballwang\anaconda3.3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
new_attribute_value = []
for index in data.attribute_value:
    index = index.lower()
    index = re.sub(r'\s','', index)
    new_attribute_value.append(index)
data['new_attribute_value'] = new_attribute_value

In [4]:
# combine data
data1 = data[['product_id', 'brand','brand_category','description','product_full_name','details','attribute_name', 'new_attribute_value', 'file']]

In [5]:
# deduct duplicate rows
data1.drop_duplicates(inplace = True)
data1.to_csv("data1.csv", index = False)

C:\Users\ballwang\anaconda3.3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
len(data1)

69399

In [7]:
data1

,product_id,brand,brand_category,description,product_full_name,details,attribute_name,new_attribute_value,file
0,01DVPPGPMCZVXSTVDN8E2XSFN3,Albus Lumen,Bags / Tote Bags / Tote Bags,Black leather (Lamb) Open top Weighs approxim...,Sensillo leather tote,This item's measurements are:\r\nDepth 0.5cm\r...,occasion,daytonight,additional
1,01E5ZXWJKJQ1AQFSQY3N1FT5WG,Re/done,Unknown,"Retro-inspired yet current, this stretchy, rib...",60s Rib Bodysuit,"True to size. XS=0-2, S=4-6, M=8-10, L=12.\r\n...",occasion,weekend,additional
2,01E5ZXWJKJQ1AQFSQY3N1FT5WG,Re/done,Unknown,"Retro-inspired yet current, this stretchy, rib...",60s Rib Bodysuit,"True to size. XS=0-2, S=4-6, M=8-10, L=12.\r\n...",occasion,daytonight,additional
3,01DVBTEVG1Y5T5DFK6MWKHM13B,ATM Anthony Thomas Melillo,Clothing / Tops / T-Shirts,Black stretch-Pima cotton jersey Slips on 94% ...,Stretch-Pima cotton jersey T-shirt,"Fits true to size, take your normal size\r\nCu...",occasion,weekend,additional
4,01DVPAKV4ANX7BXT77G0F0EART,Sally LaPointe,women:CLOTHING:PANTS,"Crafted from supple leather, Sally LaPointe's ...",Belted Leather Wide-Leg Pants,As seen on the Fall '19 runway\r\nRemovable wa...,occasion,weekend,additional
...,...,...,...,...,...,...,...,...,...
92025,01DSP0EHJP3KEWQJDC6Q8MTBCS,Movado,JewelryAccessories/Watches/ForHer/Contemporary...,From the Connect Collection. Sleek ion-plated ...,Connect 2.0 Ion-Plated Rose Gold and Black Sta...,Digital Smart module\r\nStationary black bezel...,,,
92026,01DSP0EJSPGQVG4X58MDJWDQTP,Ralph Lauren,"JustKids/Boys220/ToddlerBoys24/Bottoms,JustKid...",Dip-dyed cotton chino combines with frayed hem...,Little Boy's & Boy's Straight-Fit Dip-Dyed Cot...,Belt loops\r\nButton closure\r\nZip fly\r\nFro...,,,
92027,01DSP0ENYESDKF0FRRV3EP3QJ4,L'Objet,HomeHiTech/HG2019OLDDNU/DiningEntertaining/Din...,"From the Alchimie Collection, this bowl has an...",Alchimie Coupe Bowl,"Depth, about 12""\r\nEarthenware\r\nDishwasher ...",,,
92028,01DSP0EQ3R35KP5C7XN4C93YZS,Bonobos,TheMensStore/Apparel/Pants/Dress,,Weekday Warrior Slim-Fit Pants,Cotton dress pants that matched your formal we...,,,


# Description Feature Pre-Processing (Regex)

In [8]:
nosymbol_description = []
# find out all the percentage material in descriptions
for sent in data1.description:
    nosymbol_description.append(re.sub(r'[^\w+0-9%.]', ' ', sent, flags = re.IGNORECASE))

In [9]:
# this create substitute terms based on the percentage material
all_m = pd.Series(nosymbol_description).str.findall(r'(\d+)(?:%)\s(\w+)')
material = []
temp_content = ''
temp = []
number = 0
for i in range(len(all_m)):
    if all_m[i] != []:
        for j in all_m[i]:
            for k in j:
                if k == j[0]:
                    number = int(k)
                    if number >= 80:
                        temp_content = temp_content + "HIGH_"
                    elif number >= 40:
                        temp_content = temp_content + "MEDIAN_"
                    else:
                        temp_content = temp_content + "LOW_"
                if k == j[-1]:
                    temp_content = temp_content + k.upper()
            
            temp.append(temp_content)
            temp_content = ''
        material.append(temp)
        temp = []
    else:
         material.append('')

In [10]:
# replace all the percentage content as MATERIAL
material_description = []
for i in data1.description:
    material_description.append(re.sub(r'(\d+)(?:%)\s(\w+)', 'MATERIAL', i, flags = re.IGNORECASE))

In [11]:
# find out where the MATERIAL terms locate in each document and replace them with substitute terms created earlier
count = 0
cleaned_material = []
for i in range(len(material_description)):
    new_word = []
    for j in material_description[i].split(' '):
        if j == 'MATERIAL':
            new_word.append(material[i][count])
            count += 1
        else:
            new_word.append(j)
    count = 0
    clean = " ".join(new_word)
    cleaned_material.append(clean)         

In [12]:
no_numeric_description = []
for i in cleaned_material:
    no_numeric_description.append(re.sub(r'((\d+\.\d+)|(\d+))', 'NUMERICAL_VALUE ', i, flags = re.IGNORECASE))

In [13]:
def word_count(lines, delimiter = " "):
    words = Counter()
    for line in lines:
        line = line.lower()
        for word in line.split(delimiter):
            words[word] += 1
    return words

In [14]:
count = word_count(no_numeric_description)
count_sort = sorted((value, key) for (key, value) in count.items())[::-1]

In [15]:
# find out the difference between the 'stopwords' list and most frequent words
stopword = set(stopwords.words('english'))
count_sorted_set = set(pd.DataFrame(count_sort[:500])[1])
count_sorted_set.difference_update(stopword)
count_sorted_set

{'',
 '\r\n',
 '\r\n\r\nbody:',
 '"',
 '")',
 '&',
 "'",
 "'numerical_value",
 '(numerical_value',
 ',',
 '-',
 '/numerical_value',
 'across',
 'add',
 'added',
 'adds',
 'adjustable',
 'allover',
 'along',
 'also',
 'ankle',
 'approximately',
 'around',
 'available',
 'back',
 'back.',
 'bag',
 'belt',
 'best',
 'black',
 'blazer',
 'blend',
 'blouse',
 'blue',
 'body',
 'bold',
 'boot',
 'boots',
 "brand's",
 'breathable',
 'bring',
 'brings',
 'button',
 'button-down',
 'buttons',
 'care',
 'cashmere',
 'casual',
 'center',
 'chest',
 'chic',
 'chunky',
 'classic',
 'clean',
 'closure',
 'closure.',
 'cm',
 'coat',
 'collar',
 'collar.',
 'collection',
 'collection.',
 'color',
 'color:',
 'colors',
 'comes',
 'comfort',
 'comfort.',
 'comfortable',
 'comfy',
 'complete',
 'concealed',
 'construction',
 'contrast',
 'contrasting',
 'cool',
 'cotton',
 'cotton-blend',
 'cotton.',
 'cozy',
 'crafted',
 'create',
 'crepe',
 'crew',
 'crewneck',
 'crisp',
 'cropped',
 'crossbody',
 'cuf

In [16]:
# update the 'stopwords' list
updated_stopwords = stopwords.words('english') + [
    '.' , '', 
]

In [17]:
cleaned_descriptions = []
updated_stopwords = set(updated_stopwords)

for line in no_numeric_description:
    new_words = []
    line = re.findall(r'\b[a-zA-Z0-9_]{2,}\b',line)
    for word in line:
        if word.lower() in updated_stopwords:
            continue
        new_words.append(word.lower())
    cleaned_description = " ".join(new_words)
    cleaned_descriptions.append(cleaned_description)
    

# Brand Feature Pre-Processing (Regex)

In [18]:
def removeSpecialChars(z):
    
    return z.translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./'<>?\|`~-=_+"})

def initial_clean(data,column):
    
    data[column]=data[column].apply(str)
    data[column]=data[column].apply(lambda x:x.lower())
    data[column]=data[column].apply(lambda x:re.sub("[\(\[].*?[\)\]]", "", x))
    data[column]=data[column].apply(lambda x:removeSpecialChars(x))
    
initial_clean(data1,'brand')

C:\Users\ballwang\anaconda3.3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\ballwang\anaconda3.3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ballwang\anaconda3.3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [19]:
brand = data1['brand'].fillna(" ")
stopwords = ['by','and','the','of','for','to','inc','','a','an']
def remove_stopwords(text: str):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = list(filter(lambda token: token not in stopwords, tokens))
    return " ".join(filtered_tokens)
brand = brand.apply(remove_stopwords)

In [20]:
brand

0                       albus lumen
1                            redone
2                            redone
3        atm anthony thomas melillo
4                    sally lapointe
                    ...            
92025                        movado
92026                  ralph lauren
92027                        lobjet
92028                       bonobos
92029                      tom ford
Name: brand, Length: 69399, dtype: object

# Details Feature Pre-Processsing (Regex)

In [21]:
#get required features and fill in null values with blank
df1=data1["details"]
df1=df1.fillna(" ")

In [22]:
#remove punctuations and numbers
df1=df1.apply(lambda x: x.lower()).str.replace(r"[^a-z\s]+", "", case=False)
df1=df1.str.replace(r"\r\n", " ", case=False)
df1

0        this items measurements are depth cm height cm...
1        true to size xs s m l snap closure crewneck lo...
2        true to size xs s m l snap closure crewneck lo...
3        fits true to size take your normal size cut fo...
4        as seen on the fall  runway removable waist un...
                               ...                        
92025    digital smart module stationary black bezel go...
92026    belt loops button closure zip fly front angled...
92027    depth about  earthenware dishwasher safe made ...
92028    cotton dress pants that matched your formal we...
92029     uv protection gradient lenses cloth and case ...
Name: details, Length: 69399, dtype: object

In [23]:
count=word_count(list(df1))
count_sort=sorted((value, key) for (key, value) in count.items())[::-1]

In [24]:
from nltk.corpus import stopwords
nltk.download("stopwords")
# set(stopwords.words("English"))
nltk_stopwords = list(set(stopwords.words("English")))
count_sorted_set = set(pd.DataFrame(count_sort[:100])[1])
count_sorted_set.difference_update(nltk_stopwords)
nltk_stopwords.extend(["","h","l","mm","w","x","xl","xs","cm","uv"])

def remove_stopwords(title: str):
    tokens = nltk.word_tokenize(title)
    filtered_tokens = list(filter(lambda token: token not in nltk_stopwords, tokens))
    return " ".join(filtered_tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ballwang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
df1=df1.apply(lambda x: remove_stopwords(x))
df1

0                    items measurements depth height width
1        true size snap closure crewneck long sleeves m...
2        true size snap closure crewneck long sleeves m...
3        fits true size take normal size cut slim fit l...
4        seen fall runway removable waist unlined butto...
                               ...                        
92025    digital smart module stationary black bezel go...
92026    belt loops button closure zip fly front angled...
92027      depth earthenware dishwasher safe made portugal
92028    cotton dress pants matched formal wear belt lo...
92029    protection gradient lenses cloth case included...
Name: details, Length: 69399, dtype: object

# Product Name Pre-Processing (Regex)

In [26]:
# Make everything lowercase
product_full_name = data1['product_full_name'].str.lower()
# Remove hyphen
product_full_name = product_full_name.str.replace("-","",case=False)
# Remove numbers
product_full_name = product_full_name.str.replace(r'[0-9]+',"",case=False)
# Remove special characters and stop words
import nltk
stopwords = ['&','|',',','.',"'",'in','le','mm','and','with','the','k','x','les','de','of',"'s"]
def remove_stopwords(text: str):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = list(filter(lambda token: token not in stopwords, tokens))
    return " ".join(filtered_tokens)
product_full_name = product_full_name.apply(remove_stopwords)

# Lemmatization

In [27]:
lemmatizer = WordNetLemmatizer()
#function to convert nltk tag to wordnet tag
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wn_tagged = map(lambda x :(x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word,tag))
    return " ".join(res_words)

In [28]:
cleaned_description_lem = []
for sentence in cleaned_descriptions:
    cleaned_description_lem = cleaned_description_lem + [lemmatize_sentence(sentence)]

In [29]:
cleaned_brand_lem = []
for sentence in list(brand):
    cleaned_brand_lem = cleaned_brand_lem + [lemmatize_sentence(sentence)]

In [30]:
cleaned_details_lem = []
for sentence in list(df1):
    cleaned_details_lem = cleaned_details_lem + [lemmatize_sentence(sentence)]

In [31]:
cleaned_productname_lem = []
for sentence in list(product_full_name):
    cleaned_productname_lem = cleaned_productname_lem + [lemmatize_sentence(sentence)]

# TF-IDF Score (Combined)

We decided to combine each lemmatizating feature first. Then, we computed TF-IDF scores based on the combining document file

In [32]:
combine_lem = pd.Series(cleaned_description_lem)+" "+pd.Series(cleaned_brand_lem)+" "+\
        pd.Series(cleaned_details_lem)+" "+pd.Series(cleaned_productname_lem)


In [34]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),
                             token_pattern=r'\b[a-zA-Z0-9]{2,}\b',
                             max_df=0.8,
                             min_df = 0.01, stop_words=updated_stopwords, max_features = 500, use_idf = True)

In [42]:
X = vectorizer.fit_transform(combine_lem)
terms = vectorizer.get_feature_names()
combine_lem_tfidf_score = pd.DataFrame(X.toarray(), columns=terms)
combine_lem_idf_score = vectorizer.idf_

In [49]:
# The purpose of figuring out IDF score is for new input data to figure out TF-IDF score
combine_lem_idf_score = pd.DataFrame(combine_lem_idf_score)
combine_lem_idf_score.rename(index = {0:'idf_score'}, inplace = True)
combine_lem_idf_score.rename(columns = dict(zip(combine_lem_idf_score.columns,combine_lem_tfidf_score.columns)), inplace = True)
combine_lem_tfidf_score = combine_lem_tfidf_score.append(combine_lem_idf_score)

In [64]:
combine_lem_tfidf_score.to_csv('combine_lem_tfidf_score.csv')

In [ ]:
#combine_lem_idf_score.to_csv('idf.csv')

In [ ]:
#combine_lem_tfidf_score.to_csv('combine_lem_tfidf_score.csv')

In [ ]:
#combine_stemm_tfidf_score.to_csv('combine_stemm_tfidf_score.csv')

# Word2Vec (Combined)

We computed the Word2Vec by using combined lemmatization document file

In [51]:
docs = [word_tokenize(sentence) for sentence in combine_lem ]
combine_lem_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 500)

In [53]:
combine_lem_word2vec_model.save('combine_lem_word2vec_model.model')

# Lem Weighting Scheme (TF-IDF) 

In [54]:
X = vectorizer.fit_transform(combine_lem)
tf_idf_lookup_table = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [55]:
DOCUMENT_SUM_COLUMN = "DOCUMENT_TF_IDF_SUM"

tf_idf_lookup_table[DOCUMENT_SUM_COLUMN] = tf_idf_lookup_table.sum(axis=1)
available_tf_idf_scores = tf_idf_lookup_table.columns # a list of all the columns we have
available_tf_idf_scores = list(map(lambda x: x.lower(), available_tf_idf_scores)) # lowercase everything

In [56]:
# load spacy en_core_web_md model
nlp = spacy.load("en_core_web_md")

weighted_vectors = []
for idx, item in enumerate(combine_lem): # iterate through each product
 
    tokens = nlp(item) # have spacy tokenize the product text
    # initially start a running total of tf-idf scores for a document
    total_tf_idf_score_per_document = 0
    
    # start a running total of initially all zeroes (300 is picked since that is the word embedding size used by word2vec)
    running_total_word_embedding = np.zeros(300) 
    for token in tokens: # iterate through each token
    # if the token has a pretrained word embedding it also has a tf-idf score
        if token.has_vector and token.text.lower() in available_tf_idf_scores:
            tf_idf_score = tf_idf_lookup_table.loc[idx, token.text.lower()]
            #print(f"{token} has tf-idf score of {tf_idf_lookup_table.loc[idx, token.text.lower()]}")
            running_total_word_embedding += tf_idf_score * token.vector
            total_tf_idf_score_per_document += tf_idf_score
    
    # divide the total embedding by the total tf-idf score for each document
    document_embedding = running_total_word_embedding / total_tf_idf_score_per_document
    weighted_vectors.append(document_embedding)

C:\Users\ballwang\anaconda3.3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [57]:
#pd.DataFrame(np.array(weighted_vectors).reshape(68326, 300)).to_csv('weighted vector.csv')

# Sentence Vector (Average)

In [58]:
combine_lem = list(combine_lem)

In [59]:
docs = [word_tokenize(sentence) for sentence in combine_lem]
model = Word2Vec(docs, min_count = 1, window = 5, size = 500)

In [60]:
product_vector = []
for item in combine_lem:
    temp_vector = np.zeros((1,500))
    n = 0
    for word in item.split(" "):
        if word in model.wv.vocab:
            temp_vector += model.wv.get_vector(word)
        else:
            temp_vector += np.random.normal(size=500)
        n += 1
    product_vector.append(temp_vector/n)

In [61]:
#pd.DataFrame(np.array(product_vector).reshape(68326, 500)).to_csv('sentece vector.csv')